In [8]:
import pandas as pd
import numpy as np
from sklearn.linear_model import RidgeCV, LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold, cross_val_score
from tqdm.notebook import tqdm
import json
from rca import process_categorical, best_logistic_solver, make_binary_scoring, make_multiclass_scoring, checker
import pickle

## Loading Data

In [9]:
rca = pd.read_csv('../../data/results/rca.csv').dropna()
meta = pd.read_csv('../../data/psychNorms/psychNorms_metadata_processed.csv', index_col=0)
norms = pd.read_csv('../../data/psychNorms/psychNorms_processed.zip', index_col=0, compression='zip', low_memory=False)

# Adding norm_cat to rca
rca['norm_cat'] = (
    rca['norm'].apply(lambda norm: meta.loc[norm]['category'])
    .replace({'_': ' '}, regex=True)
)

rca

,embed,embed_type,norm,train_n,test_n,p,r2_mean,r2_sd,mse_mean,mse_sd,check,norm_cat
27,CBOW_GoogleNews,text,n_senses_wordsmyth_rice,424,107,300,0.216501,0.054134,0.021787,0.002832,pass,semantic diversity
30,CBOW_GoogleNews,text,n_features_buchanan,3501,876,300,0.155149,0.014144,0.012628,0.000561,pass,number of features
43,CBOW_GoogleNews,text,child_body_object_interact,2674,669,300,0.529288,0.018010,0.926229,0.033853,pass,motor
54,CBOW_GoogleNews,text,valence_older_kyrolainen,2855,714,300,0.668445,0.017027,0.703126,0.034586,pass,valence
55,CBOW_GoogleNews,text,valence_younger_kyrolainen,2855,714,300,0.692077,0.019715,0.739597,0.047827,pass,valence
...,...,...,...,...,...,...,...,...,...,...,...,...
6979,THINGS,behavior,imageability_vanarsdall,376,95,49,0.074112,0.090266,1317.411401,292.505564,pass,imageability
6980,THINGS,behavior,familiarity_fear,173,44,49,0.151637,0.158767,0.782185,0.220500,pass,familiarity
6981,THINGS,behavior,aoa_fear,173,44,49,0.015012,0.129597,0.514519,0.043528,pass,age of acquisition
6982,THINGS,behavior,imageability_fear,173,44,49,-0.023230,0.101781,0.262705,0.049417,pass,imageability


In [10]:
embed_avgs = (
    rca[['embed', 'norm_cat', 'r2_mean']]
    .groupby(['embed', 'norm_cat']).median(numeric_only=True) # median is used to mitigate outliers within norm_cats
    .groupby('embed').mean()
    .rename(columns={'r2_mean': 'r2_avg'})
)
embed_avgs

,r2_avg
embed,
CBOW_GoogleNews,0.455612
EEG_speech,-0.066478
EEG_text,-0.047907
GloVe_CommonCrawl,0.358081
GloVe_Twitter,0.309783
GloVe_Wikipedia,0.316517
LexVec_CommonCrawl,0.351391
PPMI_SVD_EAT,0.296195
PPMI_SVD_SWOW,0.397373


In [11]:
# Adding embed types
with open('../../data/embed_to_dtype.json', 'r') as f:
    embed_to_type = json.load(f)
embed_avgs['type'] = embed_avgs.index.map(embed_to_type)

# Finding top 2 text 
text_name_1, text_name_2 = (
    embed_avgs.query('type == "text"').sort_values('r2_avg', ascending=False).head(2).index.tolist()
)
text_name_1, text_name_2

('CBOW_GoogleNews', 'fastText_CommonCrawl')

In [12]:
# Finding top behavior
behavior_name = (
    embed_avgs.query('type == "behavior"').sort_values('r2_avg', ascending=False).head(1).index[0]
)
behavior_name

'PPMI_SVD_SWOW'

In [13]:
# Loading embeds
text_1 = pd.read_csv(f'../../data/embeds/{text_name_1}.csv', index_col=0)
text_2 = pd.read_csv(f'../../data/embeds/{text_name_2}.csv', index_col=0)
behavior = pd.read_csv(f'../../data/embeds/{behavior_name}.csv', index_col=0)

with open('../../data/brain_behav_union.pkl', 'rb') as f:
    brain_behav_union = pickle.load(f)

# Subsdetting to brain and behavior union and aligning vocabs
intersect = sorted(list(set.intersection(
    set(text_1.index), set(text_2.index), set(behavior.index), set(brain_behav_union)
)))
text_1, text_2, behavior = text_1.loc[intersect], text_2.loc[intersect], behavior.loc[intersect]

# Standardizing
standardize = lambda df: (df - df.mean()) / df.std()
text_1, text_2, behavior = standardize(text_1), standardize(text_2), standardize(behavior)

# Ensembling for comparison
embeds = {
    behavior_name: behavior,
    text_name_1: text_1, 
    text_name_2: text_2,
    text_name_1 + '&' + text_name_2: pd.concat([text_1, text_2], axis=1),
    text_name_1 + '&' + behavior_name: pd.concat([text_1, behavior], axis=1),
    text_name_2 + '&' + behavior_name: pd.concat([text_2, behavior], axis=1)
}

# Fixing column names
for embed_name, embed in embeds.items():
    embed.columns = list(range(embed.shape[1]))
    embeds[embed_name] = embed

{name: embed.shape for name, embed in embeds.items()}

{'PPMI_SVD_SWOW': (11723, 300),
 'CBOW_GoogleNews': (11723, 300),
 'fastText_CommonCrawl': (11723, 300),
 'CBOW_GoogleNews&fastText_CommonCrawl': (11723, 600),
 'CBOW_GoogleNews&PPMI_SVD_SWOW': (11723, 600),
 'fastText_CommonCrawl&PPMI_SVD_SWOW': (11723, 600)}

## Cross Validation

In [17]:
# Changing associated_embed to more usable format
meta['associated_embed'] = meta['associated_embed'].str.split(' ')

# Ridge
min_ord, max_ord = -5, 5
alphas = np.logspace(
    min_ord, max_ord, max_ord - min_ord + 1
)
ridge = RidgeCV(alphas=alphas)

# Logistic hyperparameters
Cs = 1 / alphas
inner_cv = 5
penalty = 'l2'

# Scorers
binary_scoring = make_binary_scoring()
multiclass_scoring = make_multiclass_scoring()
continuous_scoring = 'r2'

# outer_cv setting 
outer_cv, n_jobs = 5, 6

solo_embed_names = [text_name_1, text_name_2, behavior_name] # For checking data leakage in checker

# RCA
rca = []
for norm_name in tqdm(norms.columns):
    print(f'{norm_name}:')
    y = norms[norm_name].dropna()
    
    to_print = []
    for embed_name, embed in embeds.items():
        
        # Aligning embed with norm
        X, y = embed.align(y, axis='index', join='inner', copy=True)
        
        # Checking norm dtype 
        norm_dtype = meta.loc[norm_name, 'type']
        
        # Solvers, scoring, estimators ir categorical or continuous
        if norm_dtype in ['binary', 'multiclass']: # categorical
            X, y = process_categorical(outer_cv, inner_cv, X, y)
            
            # may have switched form multi to bin after processing
            norm_dtype = 'binary' if len(y.unique()) == 2 else 'multiclass'
            
            # Cross validation settings for logistic regression
            solver = best_logistic_solver(y, norm_dtype)
            
            # Defining logistic regression 
            estimator = LogisticRegressionCV(
                Cs=Cs, penalty=penalty, cv=StratifiedKFold(inner_cv), solver=solver
            )
            scoring = binary_scoring if norm_dtype == 'binary' else multiclass_scoring
        else: # continuous
            estimator, scoring = ridge, continuous_scoring
            
        # Cross validation
        associated_embed = meta.loc[norm_name, 'associated_embed']
        check = checker(solo_embed_names, y, norm_dtype, associated_embed, outer_cv)
        if check == 'pass':
            r2s = cross_val_score(estimator, X, y, cv=outer_cv, scoring=scoring, n_jobs=n_jobs) # stratification is automatically used for classification
        else:
            r2s = pd.Series([np.nan] * outer_cv)
            
        # Saving
        train_n = int(((outer_cv - 1) / outer_cv) * len(y))
        for i, r2 in enumerate(r2s):
            rca.append([embed_name, norm_name, train_n, i + 1, r2, check])
            
        # Printing
        to_print.append([embed_name, r2s.mean(), r2s.std(), check])
    to_print = pd.DataFrame(to_print, columns=['embed', 'r2_mean', 'r2_std', 'check'])
    print(to_print.sort_values('r2_mean', ascending=False).head(10).reset_index(drop=True))
    print('--------------------------------')
 
 
rca = pd.DataFrame(
    rca, columns=[
        'embed', 'norm', 'train_n', 'fold', 'r2', 'check']
)
rca.to_csv('../../data/results/rca_ensemb.csv', index=False)
rca

  0%|          | 0/291 [00:00<?, ?it/s]

frequency_lund:
                                  embed   r2_mean    r2_std check
0    fastText_CommonCrawl&PPMI_SVD_SWOW  0.829188  0.010604  pass
1  CBOW_GoogleNews&fastText_CommonCrawl  0.816586  0.013173  pass
2                  fastText_CommonCrawl  0.801812  0.012247  pass
3         CBOW_GoogleNews&PPMI_SVD_SWOW  0.723285  0.013075  pass
4                       CBOW_GoogleNews  0.665994  0.011748  pass
5                         PPMI_SVD_SWOW  0.515268  0.026448  pass
--------------------------------
frequency_kucera:
                                  embed   r2_mean    r2_std check
0    fastText_CommonCrawl&PPMI_SVD_SWOW  0.727461  0.025132  pass
1  CBOW_GoogleNews&fastText_CommonCrawl  0.712266  0.028553  pass
2                  fastText_CommonCrawl  0.690077  0.029289  pass
3         CBOW_GoogleNews&PPMI_SVD_SWOW  0.659806  0.021880  pass
4                       CBOW_GoogleNews  0.602879  0.021317  pass
5                         PPMI_SVD_SWOW  0.474919  0.043541  pass
---------

/Users/zhussain/opt/anaconda3/envs/psychProbing/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


                                  embed   r2_mean    r2_std check
0    fastText_CommonCrawl&PPMI_SVD_SWOW  0.827205  0.013322  pass
1  CBOW_GoogleNews&fastText_CommonCrawl  0.814430  0.008977  pass
2                  fastText_CommonCrawl  0.795386  0.009894  pass
3         CBOW_GoogleNews&PPMI_SVD_SWOW  0.727505  0.017173  pass
4                       CBOW_GoogleNews  0.668204  0.014296  pass
5                         PPMI_SVD_SWOW  0.495737  0.034980  pass
--------------------------------
frequency_subtlexuk:
                                  embed   r2_mean    r2_std check
0    fastText_CommonCrawl&PPMI_SVD_SWOW  0.817380  0.011416  pass
1  CBOW_GoogleNews&fastText_CommonCrawl  0.811686  0.011213  pass
2                  fastText_CommonCrawl  0.791821  0.009244  pass
3         CBOW_GoogleNews&PPMI_SVD_SWOW  0.705525  0.019186  pass
4                       CBOW_GoogleNews  0.659962  0.021916  pass
5                         PPMI_SVD_SWOW  0.450140  0.037530  pass
----------------------

KeyboardInterrupt: 